In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv
import os

load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
class LLMState(TypedDict):
    topic: str
    outline: str
    blog_post: str


def get_gemini_model():
    model = ChatGoogleGenerativeAI(
        google_api_key=os.getenv("GOOGLE_API_KEY"),
        model="gemini-2.5-flash-lite",
        temperature=0,
    )
    return model


def generate_outline(state: LLMState) -> LLMState:
    llm = get_gemini_model()
    prompt = f"Create a detailed outline for a blog post about {state['topic']}."
    response = llm.invoke(prompt)
    state["outline"] = response.content
    return state


def generate_blog_post(state: LLMState) -> LLMState:
    llm = get_gemini_model()
    prompt = f"Write a detailed blog post based on the following outline:\n{state['outline']}"
    response = llm.invoke(prompt)
    state["blog_post"] = response.content
    return state

In [3]:
graph = StateGraph(LLMState)

graph.add_node("Generate Outline", generate_outline)
graph.add_node("Generate Blog Post", generate_blog_post)

graph.add_edge(START, "Generate Outline")
graph.add_edge("Generate Outline", "Generate Blog Post")
graph.add_edge("Generate Blog Post", END)

workflow = graph.compile()

In [4]:
final_state = workflow.invoke({"topic": "The Future of AI in Healthcare"})

In [5]:
final_state["blog_post"][
    :500
]  # Display the first 500 characters of the generated blog post

"## AI's Prescription for the Future: Transforming Healthcare as We Know It\n\nImagine a world where diseases are detected before symptoms even appear, where treatments are precisely tailored to your unique genetic makeup, and where the administrative burdens that plague our healthcare system are a distant memory. This isn't science fiction; it's the rapidly approaching reality powered by Artificial Intelligence (AI).\n\nThe healthcare landscape is currently grappling with immense challenges. We face"

In [6]:
print(final_state["outline"])

## Blog Post Outline: The Future of AI in Healthcare

**Title Options:**

* The AI Revolution in Healthcare: What's Next?
* Beyond the Hype: The Real Future of AI in Medicine
* AI's Prescription for the Future: Transforming Healthcare as We Know It
* Navigating the AI Frontier: Opportunities and Challenges in Healthcare

**I. Introduction (Hook & Overview)**

*   **A. Captivating Hook:**
    *   Start with a compelling statistic about healthcare challenges (e.g., rising costs, physician burnout, diagnostic errors).
    *   Pose a thought-provoking question about the potential of technology to solve these issues.
    *   Briefly introduce AI as a transformative force.
*   **B. Defining AI in Healthcare:**
    *   Provide a concise, non-technical definition of AI and its relevance to healthcare.
    *   Mention key AI subfields relevant to healthcare (e.g., machine learning, natural language processing, computer vision).
*   **C. Thesis Statement/Core Message:**
    *   Clearly state the